In [1]:
import pandas as pd

In [2]:
with open("La Roche-Posay Toleriane Double Repair Face Moisturizer_description.txt", encoding="utf-8") as f:
    description = f.read()
description = description.replace("\n", " ")
description = description.replace("  ", " ")
description

"La Roche-Posay Toleriane Double Repair Face Moisturizer Highlights Hydrates and maintains moisture barrier Fortifies skin with ceramides Calms skin with niacinamide Product Overview La Roche-Posay Toleriane Double Repair Face Moisturizer soothes and strengthens irritated skin. La Roche-Posay Toleriane Double Repair Face Moisturizer is a daily oil-free formula that helps restore healthy, balanced skin. This cream works to strengthen the skin's natural protective barrier, providing up to 48 hours of moisture. Glycerin and Prebiotic Thermal Water gently hydrate, while essential ceramides work to prevent moisture loss. This lightweight La Roche-Posay moisturizer also contains niacinamide, a form of vitamin B3, to soothe a red, irritated complexion. La Roche-Posay Toleriane Double Repair Face Moisturizer is perfect for sensitive skin because it is formulated without parabens, fragrance or dyes. Key Features: Provides up to 48 hours of moisture for normal to dry skin Strengthens the skin's 

In [3]:
df = pd.read_csv('reviews.csv')
df.head()

,Date,Title,Review
0,"April 13, 2024",Mi crema favorita,It's the best cream I've had in years!
1,"April 11, 2024",My Go To!,This is my go to daily morning moisturizer! It...
2,"April 10, 2024",Great,My daughter uses the cream day and night and s...
3,"April 06, 2024",very hydrating,I use this mixed with a small amount of rosehi...
4,"April 06, 2024",Irritation,I started using this moisturizer and another s...


In [4]:
features = df.columns

In [5]:
df.describe()

,Date,Title,Review
count,448,448,448
unique,394,381,448
top,"December 28, 2021",Great moisturizer,It's the best cream I've had in years!
freq,3,12,1


In [6]:
df['Date'] = pd.to_datetime(df['Date'])
df['Review'] = df['Review'].str.lower()
df['Review'] = df['Review'].str.replace('\n', ' ')
df['Title'] = df['Title'].str.lower()
df['Title'] = df['Title'].str.replace('\n', ' ')
df['Combined_Title_Review'] = df['Title'] + ' ' + df['Review']

In [7]:
df.head()

,Date,Title,Review,Combined_Title_Review
0,2024-04-13,mi crema favorita,it's the best cream i've had in years!,mi crema favorita it's the best cream i've had...
1,2024-04-11,my go to!,this is my go to daily morning moisturizer! it...,my go to! this is my go to daily morning moist...
2,2024-04-10,great,my daughter uses the cream day and night and s...,great my daughter uses the cream day and night...
3,2024-04-06,very hydrating,i use this mixed with a small amount of rosehi...,very hydrating i use this mixed with a small a...
4,2024-04-06,irritation,i started using this moisturizer and another s...,irritation i started using this moisturizer an...


In [8]:
# calculate review sentiment
from textblob import TextBlob

def get_sentiment(review):
    score = TextBlob(review).sentiment.polarity
    if score > 0:
        return 'positive'
    elif score < 0:
        return 'negative'
    else:
        return 'neutral'

df['Sentiment'] = df['Combined_Title_Review'].apply(get_sentiment)
df['Sentiment'].value_counts()

Sentiment
positive    426
negative     14
neutral       8
Name: count, dtype: int64

In [9]:
# Import the OpenAI library and create a client
from openai import OpenAI
client = OpenAI()

# Set the model to use
model = "text-embedding-3-large"

# Create a function to get the embeddings (one by one)
def get_embedding(text):
    response = client.embeddings.create(input=text, model=model)
    return response

# You will notice it takes a long time to get the embeddings for all the rows
# df['embedding'] = df['PlotSummary'].apply(lambda x: get_embedding(x).data[0].embedding)

In [10]:
## So we need to batch the embeddings to get them faster
# Create a function to get the embeddings (batch by batch)
def get_batch_embedding(list_of_texts):
    response = client.embeddings.create(input=list_of_texts, model=model)
    return response

In [11]:
# Import the token counter from openai
import tiktoken
# Create a function to get the number of tokens in a text string
def num_tokens_from_string(string: str, encoding_name: str = "cl100k_base") -> int:
    """Returns the number of tokens in a text string."""
    encoding = tiktoken.get_encoding(encoding_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens

In [12]:
# get the token count for each review title
df['title_token_count'] = df['Title'].apply(lambda x: num_tokens_from_string(x))
# group the rows into batches of 8000 tokens (the actual limit is 8192 tokens, but we leave some wiggleroom) and embed them
TOKEN_LIMIT = 8000
token_size = 0
batch = []
embeddings = []
for index, row in df.iterrows():
    if token_size + row['title_token_count'] >= TOKEN_LIMIT:
        embeddings.extend([resp.embedding for resp in get_batch_embedding(batch).data])
        batch = []
        token_size = 0
    batch.append(row['Title'])
    token_size += row['title_token_count']
embeddings.extend([resp.embedding for resp in get_batch_embedding(batch).data])
# It takes me about 4 mins. Without batching it takes more than hours.

In [13]:
df['title_embedding'] = embeddings
df.head()

,Date,Title,Review,Combined_Title_Review,Sentiment,title_token_count,title_embedding
0,2024-04-13,mi crema favorita,it's the best cream i've had in years!,mi crema favorita it's the best cream i've had...,positive,5,"[-0.007242072839289904, 0.0020796703174710274,..."
1,2024-04-11,my go to!,this is my go to daily morning moisturizer! it...,my go to! this is my go to daily morning moist...,positive,4,"[0.006269732490181923, -0.001512124203145504, ..."
2,2024-04-10,great,my daughter uses the cream day and night and s...,great my daughter uses the cream day and night...,positive,1,"[0.029036294668912888, 0.009148884564638138, -..."
3,2024-04-06,very hydrating,i use this mixed with a small amount of rosehi...,very hydrating i use this mixed with a small a...,positive,3,"[-0.0054648080840706825, 0.006760748568922281,..."
4,2024-04-06,irritation,i started using this moisturizer and another s...,irritation i started using this moisturizer an...,positive,3,"[-0.026872899383306503, -0.021509340032935143,..."


In [14]:
# get the token count for each review summary
df['review_token_count'] = df['Review'].apply(lambda x: num_tokens_from_string(x))

# group the rows into batches of 8000 tokens (the actual limit is 8192 tokens, but we leave some wiggleroom) and embed them
TOKEN_LIMIT = 8000
token_size = 0
batch = []
embeddings = []
for index, row in df.iterrows():
    if token_size + row['review_token_count'] >= TOKEN_LIMIT:
        embeddings.extend([resp.embedding for resp in get_batch_embedding(batch).data])
        batch = []
        token_size = 0
    batch.append(row['Review'])
    token_size += row['review_token_count']
embeddings.extend([resp.embedding for resp in get_batch_embedding(batch).data])
# It takes me about 4 mins. Without batching it takes more than hours.

In [15]:
df['review_embeddings'] = embeddings
df.head()

,Date,Title,Review,Combined_Title_Review,Sentiment,title_token_count,title_embedding,review_token_count,review_embeddings
0,2024-04-13,mi crema favorita,it's the best cream i've had in years!,mi crema favorita it's the best cream i've had...,positive,5,"[-0.007242072839289904, 0.0020796703174710274,...",11,"[-0.015624533407390118, 0.018755080178380013, ..."
1,2024-04-11,my go to!,this is my go to daily morning moisturizer! it...,my go to! this is my go to daily morning moist...,positive,4,"[0.006269732490181923, -0.001512124203145504, ...",59,"[0.01615479402244091, -0.009959815070033073, -..."
2,2024-04-10,great,my daughter uses the cream day and night and s...,great my daughter uses the cream day and night...,positive,1,"[0.029036294668912888, 0.009148884564638138, -...",24,"[-0.016059421002864838, -0.016612326726317406,..."
3,2024-04-06,very hydrating,i use this mixed with a small amount of rosehi...,very hydrating i use this mixed with a small a...,positive,3,"[-0.0054648080840706825, 0.006760748568922281,...",53,"[-0.012470331974327564, 0.006631252355873585, ..."
4,2024-04-06,irritation,i started using this moisturizer and another s...,irritation i started using this moisturizer an...,positive,3,"[-0.026872899383306503, -0.021509340032935143,...",58,"[-0.00849886517971754, -0.022348597645759583, ..."


In [16]:
# get the token count for each review summary
df['combined_token_count'] = df['Combined_Title_Review'].apply(lambda x: num_tokens_from_string(x))

# group the rows into batches of 8000 tokens (the actual limit is 8192 tokens, but we leave some wiggleroom) and embed them
TOKEN_LIMIT = 8000
token_size = 0
batch = []
embeddings = []
for index, row in df.iterrows():
    if token_size + row['combined_token_count'] >= TOKEN_LIMIT:
        embeddings.extend([resp.embedding for resp in get_batch_embedding(batch).data])
        batch = []
        token_size = 0
    batch.append(row['Combined_Title_Review'])
    token_size += row['combined_token_count']
embeddings.extend([resp.embedding for resp in get_batch_embedding(batch).data])
# It takes me about 4 mins. Without batching it takes more than hours.

In [17]:
df['combined_embeddings'] = embeddings
df.head()

,Date,Title,Review,Combined_Title_Review,Sentiment,title_token_count,title_embedding,review_token_count,review_embeddings,combined_token_count,combined_embeddings
0,2024-04-13,mi crema favorita,it's the best cream i've had in years!,mi crema favorita it's the best cream i've had...,positive,5,"[-0.007242072839289904, 0.0020796703174710274,...",11,"[-0.015624533407390118, 0.018755080178380013, ...",16,"[-0.011924905702471733, 0.028856953606009483, ..."
1,2024-04-11,my go to!,this is my go to daily morning moisturizer! it...,my go to! this is my go to daily morning moist...,positive,4,"[0.006269732490181923, -0.001512124203145504, ...",59,"[0.01615479402244091, -0.009959815070033073, -...",63,"[0.010968859307467937, -0.013760453090071678, ..."
2,2024-04-10,great,my daughter uses the cream day and night and s...,great my daughter uses the cream day and night...,positive,1,"[0.029036294668912888, 0.009148884564638138, -...",24,"[-0.016059421002864838, -0.016612326726317406,...",25,"[-0.013202541507780552, -0.013560082763433456,..."
3,2024-04-06,very hydrating,i use this mixed with a small amount of rosehi...,very hydrating i use this mixed with a small a...,positive,3,"[-0.0054648080840706825, 0.006760748568922281,...",53,"[-0.012470331974327564, 0.006631252355873585, ...",56,"[-0.015868648886680603, 0.0010592483449727297,..."
4,2024-04-06,irritation,i started using this moisturizer and another s...,irritation i started using this moisturizer an...,positive,3,"[-0.026872899383306503, -0.021509340032935143,...",58,"[-0.00849886517971754, -0.022348597645759583, ...",61,"[-0.016827985644340515, -0.01946531981229782, ..."


In [18]:
# positive keywords
keywords = "love, best, favorite, holy grail, amazing, must-have, can't live without, ride or die, top"

# negative keywords
# keywords = "hate, worst, terrible, bad, awful, disappointed, regret, never again, avoid"

## now let's ask a question, what movie has a similar plot to the trope we thought of?
# get the embedding of the keywords
embedding = get_embedding(keywords).data[0].embedding
# get the similarity of the trope with the movie plots
import numpy as np
from numpy.linalg import norm
df['similarity'] = df['combined_embeddings'].apply(lambda x: np.dot(embedding, x) / (norm(embedding) * norm(x)))
# find top 5 similar movie plots
df = df.sort_values(by='similarity', ascending=False)
df.head()

,Date,Title,Review,Combined_Title_Review,Sentiment,title_token_count,title_embedding,review_token_count,review_embeddings,combined_token_count,combined_embeddings,similarity
152,2022-07-12,my new holy grail,i love this rich moisturizer. though it feels ...,my new holy grail i love this rich moisturizer...,positive,5,"[-0.006340186111629009, -0.010606165044009686,...",50,"[0.0014807990519329906, -0.0021670230198651552...",55,"[-0.006066050846129656, -0.0035829090047627687...",0.381254
62,2023-07-16,"love, love, love",wonderful moisturizer. this soaks right in and...,"love, love, love wonderful moisturizer. this s...",positive,5,"[-0.00014493693015538156, 0.01193617470562458,...",30,"[0.005224585998803377, 0.02660480886697769, -0...",33,"[-0.002207394689321518, 0.026195747777819633, ...",0.365725
176,2022-04-04,holy grail!,dermatologist recommended! a virtual dupe for ...,holy grail! dermatologist recommended! a virtu...,positive,5,"[0.025286292657256126, -0.01726226694881916, -...",26,"[-0.014521864242851734, -0.008821570314466953,...",30,"[-0.008320685476064682, -0.010359800420701504,...",0.361132
0,2024-04-13,mi crema favorita,it's the best cream i've had in years!,mi crema favorita it's the best cream i've had...,positive,5,"[-0.007242072839289904, 0.0020796703174710274,...",11,"[-0.015624533407390118, 0.018755080178380013, ...",16,"[-0.011924905702471733, 0.028856953606009483, ...",0.358132
382,2019-10-14,love it!,"i have used many facial moisturizers, and this...","love it! i have used many facial moisturizers,...",positive,3,"[-0.02595599927008152, -0.010833550244569778, ...",18,"[-0.017888978123664856, -0.0017328480025753379...",21,"[-0.026575254276394844, -0.01647029258310795, ...",0.355073


In [19]:
#df.to_csv('reviews_with_embeddings.csv', index=False)

In [20]:
# form the context from the top 10 similar reviews
context = ""
for i in range(10):
  context += "Review Date: " + str(df.iloc[i]['Date']) + " \n " + "Title" + df.iloc[i]['Title'] + " \n " + "Review: " + df.iloc[i]['Review']
context

"Review Date: 2022-07-12 00:00:00 \n Titlemy new holy grail \n Review: i love this rich moisturizer. though it feels rich and moisturizing, it doesn't feel heavy on skin. the formulation is just perfect and it doesn't give me pimples nor any reactions. it is a must in every woman's skincare!Review Date: 2023-07-16 00:00:00 \n Titlelove, love, love \n Review: wonderful moisturizer. this soaks right in and i can layer it over other products without any problem. another great product from this brand.Review Date: 2022-04-04 00:00:00 \n Titleholy grail! \n Review: dermatologist recommended! a virtual dupe for some of the expensive moisturizers. works great for my dry, mature skin.Review Date: 2024-04-13 00:00:00 \n Titlemi crema favorita \n Review: it's the best cream i've had in years!Review Date: 2019-10-14 00:00:00 \n Titlelove it! \n Review: i have used many facial moisturizers, and this is my favorite! will buy again.Review Date: 2020-08-29 00:00:00 \n Titlemy favorite \n Review: i use

In [21]:
def send_prompt(prompt):
  # write the request to OpenAI API
  completion = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
      {"role": "system", "content": "You are a brand specialist for a company that makes skincare products with expertise in product differntiation techniques and marketing."},
      {"role": "user", "content": prompt}
    ]
  )
  print(completion.choices[0].message.content)
  return(completion.choices[0].message.content)

In [22]:
from openai import OpenAI
from IPython.display import Image

In [23]:
# Image generation using DALL·E 3
def generate_image(client, prompt, size="1024x1024", quality="standard", n=1):
    response = client.images.generate(
        model="dall-e-3",
        prompt=prompt,
        size=size,
        quality=quality,
        n=n,
    )
    return response

In [24]:
def get_image(response):
    # Create a client
    client = OpenAI()

    # Example usage
    prompt = response
    response = generate_image(client, prompt)

    image_url = response.data[0].url
    image = Image(url=image_url)
    return image, image_url

In [25]:
from IPython.display import display, Image
from PIL import Image as PILImage
import requests
from io import BytesIO

def save_image(url, file_name):

    # To save it, you would need to fetch it again:
    response = requests.get(url)
    image_bytes = BytesIO(response.content)
    pil_img = PILImage.open(image_bytes)
    pil_img.save(file_name)

In [26]:
def get_image_description(image_url):
  response = client.chat.completions.create(
  model="gpt-4-vision-preview",
  messages=[
    {
      "role": "user",
      "content": [
        {"type": "text", "text": "Give me a description of this image."},
        {
          "type": "image_url",
          "image_url": {
            "url": image_url,
          },
        },
      ],
    }
  ],
  max_tokens=300,
  )
  return response.choices[0].message.content

In [27]:
import nltk
from nltk.tokenize import sent_tokenize

def display_image_description(response):
    sentences = sent_tokenize(response)

    for sentence in sentences:
        print(sentence)




In [28]:
prompt1 = "Provide a brief summary of the product given its description, details, and the provided reviews for context. \n Description: " + description + "\n" + context
response1 = send_prompt(prompt1)
image1, image_url1 = get_image(response1)
display_image_description(get_image_description(image_url1))
#save_image(image_url1, 'product_image1.png')
print(response1)

The La Roche-Posay Toleriane Double Repair Face Moisturizer is a highly praised skincare product that is designed to hydrate, strengthen the skin's moisture barrier, and soothe red, irritated skin. It is formulated with ceramides, niacinamide, glycerin, and Prebiotic Thermal Water to provide up to 48 hours of moisture for normal to dry skin. The product is oil-free and suitable for sensitive skin as it does not contain parabens, fragrance, or dyes.

The reviews for this moisturizer are overwhelmingly positive, with customers praising its lightweight feel, effectiveness in hydrating the skin without feeling heavy, and its compatibility with other skincare products. Many users have also mentioned that it is their go-to moisturizer and have seen improvements in their skin's hydration and overall condition.

Overall, the La Roche-Posay Toleriane Double Repair Face Moisturizer appears to be a popular choice among customers for its hydrating and soothing properties, making it a potentially t

In [29]:
prompt2 = "List notable features of the product given its description, details, and the provided reviews for context. \n Description: " + description + "\n" + context
response2 = send_prompt(prompt2)
image2, image_url2 = get_image(response2)
display_image_description(get_image_description(image_url2))
#save_image(image_url2, 'product_image2.png')
print(response2)

Notable Features of La Roche-Posay Toleriane Double Repair Face Moisturizer:

1. **Hydrates and Maintains Moisture Barrier**: Provides up to 48 hours of moisture for normal to dry skin, helping to keep the skin hydrated and healthy.

2. **Fortifies Skin with Ceramides**: Contains essential ceramides that work to strengthen the skin's natural protective barrier, aiding in moisture retention and skin protection.

3. **Calms Skin with Niacinamide**: Niacinamide (vitamin B3) helps soothe red, irritated skin, making it suitable for sensitive skin types.

4. **Lightweight Formula**: Despite being rich and moisturizing, the formula does not feel heavy on the skin, making it comfortable for daily use.

5. **Non-Comedogenic**: Suitable for acne-prone skin as it is non-comedogenic, which means it won't clog pores.

6. **Fragrance-Free, Paraben-Free**: Formulated without parabens, fragrance, or dyes, making it gentle on the skin and ideal for sensitive skin.

7. **Versatile Use**: Can be used as 

In [30]:
prompt3 = "Conduct sentiment analysis of the product reviews given its description, details, and the provided reviews for context. \n Description: " + description + "\n" + context
response3 = send_prompt(prompt3)
image3, image_url3 = get_image(response3)
display_image_description(get_image_description(image_url3))
#save_image(image_url3, 'product_image3.png')
print(response3)

Based on the sentiment analysis of the provided reviews for the La Roche-Posay Toleriane Double Repair Face Moisturizer, the overall sentiment is very positive. 

1. "my new holy grail": This review expresses high satisfaction with the product, mentioning that it feels rich and moisturizing without being heavy on the skin. The reviewer considers it a must-have in their skincare routine, indicating a strong positive sentiment.

2. "love, love, love": The reviewer shows strong admiration for the moisturizer, stating that it works well with other products and is another great product from the brand. This review conveys a very positive sentiment.

3. "holy grail!": The reviewer describes the product as a dermatologist-recommended dupe for more expensive moisturizers, indicating effectiveness and value for money. The sentiment here is positive and appreciative.

4. "mi crema favorita": This Spanish review translates to "my favorite cream" and expresses high praise for the product, indicatin

In [40]:
prompt4 = "Using the provided product description, details, and reviews for context. Generate a prompt with specific product visuals to use as input to DALL-E 3 with the intention of creating an image of the product described. \n Description: " + description + "\n" + "Reviews: " + context
response4 = send_prompt(prompt4)
image4, image_url4 = get_image(response4)
display_image_description(get_image_description(image_url4))
save_image(image_url4, 'product_image4.png')
print(response4)

Create an image of a sleek, modern skincare product bottle for "La Roche-Posay Toleriane Double Repair Face Moisturizer." The bottle should feature a clean design with gentle curves and a soft matte finish to convey a sense of luxury. Include key elements such as the product name, "Toleriane Double Repair Face Moisturizer," prominently displayed on the front label. Incorporate visuals that symbolize hydration, such as subtle water droplets or a dewy glow, to emphasize the moisturizing properties of the product. Additionally, showcase the key ingredients like ceramides and niacinamide on the packaging for a scientific and advanced skincare impression. The color scheme should convey a sense of purity and effectiveness, possibly featuring shades of soft blues and greens.
The image showcases a product presentation for a facial moisturizer.
There is a large bottle to the right with a matte, dark gray finish and a pump dispenser on top.
Centered on the bottle is a label featuring the text "H

In [36]:
# positive keywords
keywords = "love, best, favorite, holy grail, amazing, must-have, can't live without, ride or die, top"

# negative keywords
# keywords = "hate, worst, terrible, bad, awful, disappointed, regret, never again, avoid"

## now let's ask a question, what movie has a similar plot to the trope we thought of?
# get the embedding of the keywords
embedding = get_embedding(keywords).data[0].embedding
# get the similarity of the trope with the movie plots
import numpy as np
from numpy.linalg import norm
df['similarity'] = df['combined_embeddings'].apply(lambda x: np.dot(embedding, x) / (norm(embedding) * norm(x)))
# find top 5 similar movie plots
df = df.sort_values(by='similarity', ascending=False)
df.head()

,Date,Title,Review,Combined_Title_Review,Sentiment,title_token_count,title_embedding,review_token_count,review_embeddings,combined_token_count,combined_embeddings,similarity
152,2022-07-12,my new holy grail,i love this rich moisturizer. though it feels ...,my new holy grail i love this rich moisturizer...,positive,5,"[-0.006340186111629009, -0.010606165044009686,...",50,"[0.0014807990519329906, -0.0021670230198651552...",55,"[-0.006066050846129656, -0.0035829090047627687...",0.381244
62,2023-07-16,"love, love, love",wonderful moisturizer. this soaks right in and...,"love, love, love wonderful moisturizer. this s...",positive,5,"[-0.00014493693015538156, 0.01193617470562458,...",30,"[0.005224585998803377, 0.02660480886697769, -0...",33,"[-0.002207394689321518, 0.026195747777819633, ...",0.365731
176,2022-04-04,holy grail!,dermatologist recommended! a virtual dupe for ...,holy grail! dermatologist recommended! a virtu...,positive,5,"[0.025286292657256126, -0.01726226694881916, -...",26,"[-0.014521864242851734, -0.008821570314466953,...",30,"[-0.008320685476064682, -0.010359800420701504,...",0.361120
0,2024-04-13,mi crema favorita,it's the best cream i've had in years!,mi crema favorita it's the best cream i've had...,positive,5,"[-0.007242072839289904, 0.0020796703174710274,...",11,"[-0.015624533407390118, 0.018755080178380013, ...",16,"[-0.011924905702471733, 0.028856953606009483, ...",0.358150
382,2019-10-14,love it!,"i have used many facial moisturizers, and this...","love it! i have used many facial moisturizers,...",positive,3,"[-0.02595599927008152, -0.010833550244569778, ...",18,"[-0.017888978123664856, -0.0017328480025753379...",21,"[-0.026575254276394844, -0.01647029258310795, ...",0.355068


In [37]:
#df.to_csv('reviews_with_embeddings.csv', index=False)

In [38]:
# form the context from the top 10 similar reviews
context = ""
for i in range(10):
  context += "Review Date: " + str(df.iloc[i]['Date']) + " \n " + "Title" + df.iloc[i]['Title'] + " \n " + "Review: " + df.iloc[i]['Review']
context

"Review Date: 2022-07-12 00:00:00 \n Titlemy new holy grail \n Review: i love this rich moisturizer. though it feels rich and moisturizing, it doesn't feel heavy on skin. the formulation is just perfect and it doesn't give me pimples nor any reactions. it is a must in every woman's skincare!Review Date: 2023-07-16 00:00:00 \n Titlelove, love, love \n Review: wonderful moisturizer. this soaks right in and i can layer it over other products without any problem. another great product from this brand.Review Date: 2022-04-04 00:00:00 \n Titleholy grail! \n Review: dermatologist recommended! a virtual dupe for some of the expensive moisturizers. works great for my dry, mature skin.Review Date: 2024-04-13 00:00:00 \n Titlemi crema favorita \n Review: it's the best cream i've had in years!Review Date: 2019-10-14 00:00:00 \n Titlelove it! \n Review: i have used many facial moisturizers, and this is my favorite! will buy again.Review Date: 2020-08-29 00:00:00 \n Titlemy favorite \n Review: i use

In [35]:
prompt5 = "Generate a list of product details to highlight for this product in a ad campaign according to negative reviews of the product and its description. \n Description: " + description + "\n" + "Reviews: " + context
response5 = send_prompt(prompt5)
image5, image_url5 = get_image(response5)
display_image_description(get_image_description(image_url5))
save_image(image_url5, 'product_image5.png')
print(response5)

Based on the negative reviews and the product description, here are some product details to highlight in an ad campaign to address customer concerns and improve perception:

1. Non-comedogenic formula: Highlight that the La Roche-Posay Toleriane Double Repair Face Moisturizer is non-comedogenic, making it suitable for sensitive and acne-prone skin, and helps prevent clogged pores and breakouts.
   
2. Fragrance-Free: Emphasize that the moisturizer is free from fragrance, appealing to customers who are sensitive to scents and prefer products without added fragrances.

3. Lightweight Texture: Mention that the moisturizer has a lightweight formula, ensuring quick absorption and non-greasy finish, ideal for daily use and suitable for both morning and evening application.

4. Reduces Irritation: Focus on the inclusion of niacinamide, a calming ingredient that helps soothe redness and irritation, making it suitable for those with sensitive or reactive skin.

5. Nourishing Ceramides: Highligh